In [1]:
import numpy as np
import pandas as pd
from itertools import product
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator

import gurobipy as grb
from gurobipy import GRB

# Input Problem Parameters

## Import du problème

In [2]:
#import define_nums
from utils.extract_parameters import extract_parameters
from utils.define_nums import define_nums
from utils.domination_3D import liste_dominants, graph_dominant

In [3]:
################################
######## TO DO #################
################################

# choose taille among: "small", "medium", "large"
taille = "small"

In [4]:
################################
######## DO NOT CHANGE #########
################################
map_employe_indice, map_job_indice, map_qualif_indice, conge, qualif, d, p, b, besoin = extract_parameters(taille)
nombre_employes, horizon, nombre_qualifs, nombre_projets = define_nums(conge, qualif, d, p, b, besoin)

## Définissons les paramètres

In [5]:
## u_j, le projet j est réalisé
u_shape = nombre_projets

## x_i_j_k_n, l'employé k travaille sur le projet j avec la qualif i
## au jour n
x_shape = [(i,j,k, n) for i in range(nombre_qualifs) for j in range(nombre_projets) for k in range(nombre_employes) for n in range(horizon)]

## t_k_j, l'employé k travaille sur le projet j
t_shape = [(i,j) for i in range(nombre_employes) for j in range(nombre_projets)]

## begin_j, la date de début effective
begin_shape = nombre_projets

## end_j, la date de fin effective = date de livraison du projet j
end_shape = nombre_projets

## delay_j, le nombre de jours de retard par projet j
delay_shape = nombre_projets

## longueur_j, la longueur du projet j
longueur_shape = nombre_projets

## projet_par_employe_k, le nombre de projet par employe
projet_par_employe_shape = nombre_employes

# majorant pour la contrainte de couverture
maj = max([besoin[i][j] for j in range(nombre_projets) for i in range(nombre_qualifs)]) + 1

# si le projet j est travaillé au jour n
job_in_process_shape = [(j, n) for j in range(nombre_projets) for n in range(horizon)]

# Model deployment

## Add the variables

In [6]:
model = grb.Model("Planning")

# L'affectation du planning, la variable clé
x = model.addVars(x_shape, vtype=GRB.BINARY, name="x")

## Les dates de fin des projets
dateFin = model.addVars(end_shape, lb = 0, ub = horizon, name = "dateFin")

## Les dates de début des projets
dateDebut = model.addVars(begin_shape, lb = 0, ub = horizon, name = "dateDebut")

## Le nombre de jours de delai
delay = model.addVars(delay_shape, lb = 0, ub = horizon, name = "delay")

## Les longueurs des projets
longueur = model.addVars(longueur_shape, lb = 0, ub = horizon, name = "longueur")

## Nombre de projet par employé
projet_par_employe = model.addVars(projet_par_employe_shape, lb = 0, ub = nombre_projets, name = "projet_par_employe")

## L'affectation sur un projet d'un employé
affectation = model.addVars(t_shape, vtype = GRB.BINARY, name = "affectation")

## La réalisation ou non d'un projet
u = model.addVars(u_shape, vtype = GRB.BINARY, name="u")

## Verification du projet
job_in_process = model.addVars(job_in_process_shape, vtype = GRB.BINARY, name = "job_in_process")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-29


In [7]:
# Update le modele pour confirmer la création des variables
model.update()

## Add constraints

In [8]:
for i in range(nombre_qualifs):
    for j in range(nombre_projets):
        for k in range(nombre_employes):
            for n in range(horizon):
                ## La qualification du personnel
                model.addConstr( x[i,j,k,n] <= qualif[i][k], name="QualificationPersonnel[{0}{1}{2}{3}]".format(i,j,k,n))
                model.addConstr( x[i,j,k,n] <= (besoin[i][j] >=1), name = "BesoinQualifPersonnel[{0}{1}{2}{3}]".format(i,j,k,n))
                ## Contraintes des congés
                model.addConstr( x[i,j,k,n] <= conge[k][n], name=f"CongesPersonnel[{i}{j}{k}{n}]")
        
        ## Contrainte d'unicité de réalisation d'un projet
        model.addConstr(grb.quicksum(grb.quicksum(x[i,j,k,n] for n in range(horizon)) for k in range(nombre_employes)) == besoin[i][j]*u[j], name=f"UniciteRealisation[{i}{j}]")


## Unicité de l'affectation quotidienne
for k in range(nombre_employes):
    for n in range(horizon):
        model.addConstr(grb.quicksum(grb.quicksum(x[i,j,k,n] for j in range(nombre_projets)) for i in range(nombre_qualifs)) <= 1, name=f"UniciteAffectation[{k}{n}]")

        
for j in range(nombre_projets):
    for n in range(horizon):
        model.addConstrs(((job_in_process[j,n] >= x[i,j,k,n]) for i in range(nombre_qualifs) for k in range(nombre_employes)), name=f"ContrainteJobInProcess[{j}{n}]")
    # start date
    model.addConstrs(((dateDebut[j] <= job_in_process[j,n]*n) for n in range(horizon)), name = f"ContrainteDateDebut[{j}]")

    # end date
    model.addConstrs(((job_in_process[j,n]*n <= dateFin[j]) for n in range(horizon)), name=f"FinProjet[{j}]")

    ## longueur
    model.addConstr(longueur[j] == dateFin[j] - dateDebut[j], name=f"LongueurProjet[{j}]")

    ## delay
    model.addConstr(delay[j] >= dateFin[j] - (d[j] - 1), name=f"DelaiProjet_difference[{j}]")    

## projet_par_employe
for k in range(nombre_employes):
    model.addConstr(projet_par_employe[k] == grb.quicksum( affectation[k,j] for j in range(nombre_projets)), name=f"FinProjet[{j}]")
    
    
## Contrainte de l'affectation à un projet selon les jours de travail
for k in range(nombre_employes):
    for j in range(nombre_projets):
        model.addConstr(affectation[k,j] <= grb.quicksum(grb.quicksum(x[i,j,k,n] for i in range(nombre_qualifs)) for n in range(horizon)), name=f"AffectationProjetQuicksum[{k}{j}]")    

## Fonctions objectif

In [9]:
## bénéfice total
benefice_tot = grb.quicksum( (b[j]*u[j] - delay[j]*p[j]) for j in range(nombre_projets) )

In [10]:
# Lignes pour optimiser le problème suivant
## les trois fonctions objectif voulues

# Nombre de projet maximum par personne
projet_max = model.addVar(lb= 0, ub=nombre_projets, name="projet_max")
model.addConstrs(projet_max>=projet_par_employe[k] for k in range(nombre_employes))

# Compacité des projets
longueur_max = model.addVar(lb = 0, ub=horizon, name="longueur_max")
model.addConstrs(longueur_max>=longueur[j] for j in range(nombre_projets))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [11]:
model.update()

In [12]:
res = pd.DataFrame(columns = ["Benefice", "longueur max", "projets max"])

for l in range(horizon-1, 0 , -1):
    for p in range(nombre_projets-1, 0, -1):
        # contrainte sur la longueur des projets
        model.addConstr(longueur_max<=l)
        # contrainte sur le nombre de projets
        model.addConstr(projet_max<=p)

        model.ModelSense = GRB.MAXIMIZE
        model.setObjective(benefice_tot)
        model.optimize()

        res.loc[len(res)] = [model.objVal, int(longueur_max.x), int(projet_max.x)]
        
        model.reset(1)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1018 rows, 295 columns and 1976 nonzeros
Model fingerprint: 0x6790b0ab
Variable types: 25 continuous, 270 integer (270 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 966 rows and 190 columns
Presolve time: 0.00s
Presolved: 52 rows, 105 columns, 270 nonzeros
Variable types: 0 continuous, 105 integer (102 binary)
Found heuristic solution: objective 20.0000000

Root relaxation: objective 7.200000e+01, 79 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

 

Variable types: 0 continuous, 78 integer (78 binary)
Found heuristic solution: objective 20.0000000

Root relaxation: objective 6.000000e+01, 49 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   60.00000    0    1   20.00000   60.00000   200%     -    0s
H    0     0                      55.0000000   60.00000  9.09%     -    0s
     0     0   60.00000    0    7   55.00000   60.00000  9.09%     -    0s
     0     0   55.00000    0    3   55.00000   55.00000  0.00%     -    0s

Cutting planes:
  Gomory: 2
  Cover: 1
  Zero half: 1

Explored 1 nodes (62 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 3: 55 20 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.500000000000e+01, best bound 5.500000000000e+01, gap 0.0000%
Discarded solution information

Presolve removed 1019 rows and 240 columns
Presolve time: 0.00s
Presolved: 17 rows, 55 columns, 110 nonzeros
Variable types: 0 continuous, 55 integer (55 binary)
Found heuristic solution: objective 20.0000000

Root relaxation: objective 4.500000e+01, 26 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   45.00000    0    5   20.00000   45.00000   125%     -    0s
H    0     0                      40.0000000   45.00000  12.5%     -    0s
     0     0   45.00000    0    5   40.00000   45.00000  12.5%     -    0s

Explored 1 nodes (30 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 3: 40 20 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+01, best bound 4.000000000000e+01, gap 0.0000%
Discarded solution information including additional


Explored 1 nodes (24 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 20 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+01, best bound 2.000000000000e+01, gap 0.0000%
Discarded solution information including additional information
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1048 rows, 295 columns and 2006 nonzeros
Model fingerprint: 0xe37cd777
Variable types: 25 continuous, 270 integer (270 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 1034 rows and 260 columns
Presolve time: 0.00s
Presolved: 14 rows, 35 columns, 70 nonzeros
Variable

# Model optimisation

In [13]:
res.to_excel("results/final/"+taille+"/res.xlsx")

In [14]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

X, Y = np.meshgrid(res["projets max"], res["longueur max"])
Z = res["Benefice"]

X = np.array(res["projets max"])
Y = np.array(res["longueur max"])
Z = np.array(res["Benefice"])

# Plot the surface.
surf = ax.plot_trisurf(X, Y, Z, cmap = 'Spectral')

# Add a color bar which maps values to colors.
fig.colorbar(surf, pad=0.2)

ax.set_xlabel('projets max')
ax.set_ylabel('longueur max')
ax.set_zlabel('bénéfice')

fig.set_size_inches(7, 5)

plt.savefig("results/sys_preference/"+taille+".png")
plt.show()

ValueError: Unknown projection '3d'

<Figure size 432x288 with 0 Axes>

In [ ]:
X = np.array(res["longueur max"])
Y = np.array(res["projets max"])
Z = np.array(res["Benefice"])

#longueur/projet
plt.scatter(X,Y,label="longueur/projet", marker='+')
plt.title("Projets max en fonction de longueur")
plt.savefig("results/graphs_biobjectifs/Domination longueur_max projet_max"+taille)
plt.show()

Sur le plot précédent, on remarque que la seule solution non dominée lorsqu'on regarde les critères longueur_max et projet_max correspond à (1,1).
Introduire un système de préférence sur ces deux critères n'est pas nécessaire.

In [ ]:
#longueur/projet
plt.scatter(Y, Z,label="Bénéfice en fonction de projets max", marker='+', c=X)
plt.title("Bénéfice en fonction de projets max")
plt.colorbar()
plt.savefig("results/graphs_biobjectifs/Domination Benefice projet_max"+taille)
plt.show()

Sur le plot précédent, on remarque que la seule solution non dominée lorsqu'on regarde les critères bénéfice et projet_max.
Introduire un système de préférence sur ces deux critères n'est pas nécessaire.

In [ ]:
#longueur/benefice
plt.scatter(X,Z,label="Bénéfice en fonction de longueur max", marker = '+', c=Y)
plt.colorbar()
plt.title("Bénéfice en fonction de longueur max")
plt.savefig("results/graphs_biobjectifs/Domination longueur_max bénéfice"+taille)
plt.show()